In [127]:
# This jupyter notebook demonstrates the use of Python 
# to generate csv output identical to that generated by
# 'geograph_demo_notebook.ipynb'. The primary benefit 
# is a function of interaction with the graph database.
# Here, only read operations are performed with cypher
# queries. This prevents erroneous mutations of your 
# local database, and removes the clean up step entirely.  

In [128]:
import pandas as pd
from tabulate import tabulate
from py2neo import Graph,Node,Relationship


In [129]:
geograph = Graph('http://localhost:7474', name='geograph')

In [130]:
# For both queries used in this notebook, the data returned 
# when the query is run is written to a pandas style dataframe 
# using a function from py2neo. This allows us to work with the 
# data in-situ instead of relying on the set intersection and 
# calculation done with the final cypher query (table_output_query 
# in geograph_demo_notebook.ipynb). 
base_view_query = '''
MATCH (s:State)-[]-(c:County)-[r]-(t:Censustract)
RETURN s.stateName AS state, c.countyName AS county, count(r) AS initial_indegree;
'''

In [131]:
filtered_view_query = '''
MATCH (s:State)-[]-(c:County)-[r]-(t:Censustract)
WHERE
(t.leakyUndergroundStorageTanksPercentile >= 90 OR t.wastewaterDischargePercentile >= 90)
 AND t.percentOfIndividualsBelow200FederalPovertyLinePercentile >= 65
 AND
(t.expectedAgriculturalLossRateNaturalHazardsRiskIndexPercentile >= 90
 OR t.expectedBuildingLossRateNaturalHazardsRiskIndexPercentile >= 90
 OR t.expectedPopulationLossRateNaturalHazardsRiskIndexPercentile >= 90
 OR t.shareOfPropertiesAtRiskOfFloodIn30YearsPercentile >= 90
 OR t.shareOfPropertiesAtRiskOfFireIn30YearsPercentile >= 90)
WITH s, c, count(r) AS indegree
RETURN s.stateName AS state, c.countyName AS county, indegree as final_indegree
'''

In [132]:
# To generate the desired csv output, the data returned by
# the base view query and the filtered view query are saved 
# to separate dataframes
base_view_df = geograph.run(base_view_query).to_data_frame()
filtered_view_df = geograph.run(filtered_view_query).to_data_frame()

In [133]:
# The base view table contains data representing the number of
# census tracts (initial_indegree) that compose each US county
# for every US state.
base_view_df

,state,county,initial_indegree
0,Alabama,Pickens County,5
1,Alabama,Sumter County,4
2,Alabama,Pike County,8
3,Alabama,Tallapoosa County,10
4,Alabama,Winston County,7
...,...,...,...
3130,Wyoming,Uinta County,3
3131,Wyoming,Sublette County,2
3132,Wyoming,Sheridan County,6
3133,Wyoming,Teton County,4


In [134]:
# The filtered view table represents counties that with census tracts  
# that meet the filtering criteria of the default CEJ water indicator and
# climate indicator. The number of such census tracts in a given county
# is the row's final_indegree value. 
filtered_view_df

,state,county,final_indegree
0,Alabama,Etowah County,2
1,Alabama,Jefferson County,8
2,Alabama,Mobile County,18
3,Alaska,Anchorage_ Orough,2
4,Alaska,Fairbanks_north_star_ Orough,1
...,...,...,...
379,West Virginia,Tyler County,1
380,West Virginia,Wayne County,2
381,West Virginia,Wetzel County,2
382,West Virginia,Wood County,1


In [135]:
# To generate a table matching the csv output by geograph_demo_notebook.ipynb
# the intersection of the state and county columns from base_view_df and
# filtered_view_df is extracted into a new merged_df using an inner join on
# the 'state' and 'county' columns. 
merged_df = base_view_df.merge(filtered_view_df, on=['state', 'county'], how='inner')
# The proportion column is created by dividing the number of census tracts
# remaining (final_indegree) by the total number of census tracts in that county
# (initial_indegree) and assigning those values to a new merged_df column, 'proportion'.
merged_df['proportion'] = merged_df['final_indegree'] / merged_df['initial_indegree']
# The final version of the table is created by selecting columns names in-order to
# match the demo notebook's csv output, these columns are assigned to final_df after
# being extracted from merged_df.  
final_df = merged_df[['state', 'county', 'initial_indegree', 'final_indegree', 'proportion']]
# In the demo notebook's final cypher query, an 'ORDER BY proportion DESC' clause 
# is appended to the return statement, which we recreate with pd.sort_values(ascending=False) . 
table_view_df = final_df.sort_values(by='proportion', ascending=False)

In [136]:
print(tabulate(table_view_df, headers='keys', tablefmt='excel'))

     state                county                          initial_indegree    final_indegree    proportion
---  -------------------  ----------------------------  ------------------  ----------------  ------------
362  West Virginia        Mingo County                                   7                 7    1
138  Kentucky             Martin County                                  3                 3    1
173  Mississippi          Tunica County                                  3                 2    0.666667
317  Texas                Kleberg County                                 6                 3    0.5
367  West Virginia        Mason County                                   6                 3    0.5
179  Missouri             Reynolds County                                2                 1    0.5
344  Virginia             Nottoway County                                4                 2    0.5
109  Illinois             Massac County                                  4           

In [137]:
output = table_view_df.reset_index(drop=True)
output.to_csv("output.csv", index=False)